In [1]:
import pandas as pd
import joblib

In [2]:
df1=pd.read_csv('train.csv')

In [3]:
df1

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,LP002978,Female,No,0,Graduate,No,2900,0.0,71.0,360.0,1.0,Rural,Y
610,LP002979,Male,Yes,3+,Graduate,No,4106,0.0,40.0,180.0,1.0,Rural,Y
611,LP002983,Male,Yes,1,Graduate,No,8072,240.0,253.0,360.0,1.0,Urban,Y
612,LP002984,Male,Yes,2,Graduate,No,7583,0.0,187.0,360.0,1.0,Urban,Y


In [4]:
df1=df1.dropna()

In [5]:
del df1['Loan_ID']
X=df1[df1.columns[0:11]]
y=df1['Loan_Status']

In [6]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [7]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
le.fit_transform(y_train)
cols=['Gender','Married','Education','Self_Employed','Dependents','Property_Area']
X_train[cols]=X_train[cols].apply(le.fit_transform)
X_test[cols]=X_test[cols].apply(le.fit_transform)

In [8]:
X_train.iloc[2]

Gender                  1.0
Married                 1.0
Dependents              2.0
Education               1.0
Self_Employed           0.0
ApplicantIncome      3987.0
CoapplicantIncome    1411.0
LoanAmount            157.0
Loan_Amount_Term      360.0
Credit_History          1.0
Property_Area           0.0
Name: 607, dtype: float64

In [9]:
from sklearn.naive_bayes import GaussianNB
modelgnb = GaussianNB()
modelgnb.fit(X_train, y_train)


GaussianNB()

In [10]:
y_pred=modelgnb.predict(X_test)

In [11]:
from sklearn.metrics import confusion_matrix,precision_score,recall_score
p=confusion_matrix(y_test,y_pred)
print("Accurarcy::",(p[0][0]+p[1][1])/(p[0][0]+p[0][1]+p[1][0]+p[1][1]))

Accurarcy:: 0.8229166666666666


In [12]:
joblib.dump(modelgnb,'Loan Approval')

['Loan Approval']

In [13]:
%%writefile app.py

import streamlit as st
import joblib as job
model=job.load("Loan Approval")
st.title("Loan Approval")
st.header("Fill the details below::")
formbtn = st.button("Form")
if "formbtn_state" not in st.session_state:
    st.session_state.formbtn_state = False

if formbtn or st.session_state.formbtn_state:
    st.session_state.formbtn_state = True
    with st.form("form"):
      ID=st.text_input("Enter your LoanID::")
      gender=st.radio("Select your Gender::",("Male","Female"))
      if gender=='Male':
        gender=1
      else:
        gender=0
      mar=st.radio("Select your Marital Status::",("No","Yes"))
      if mar=="Yes":
        mar=1
      else:
        mar=0
      dep=st.selectbox("Number of Dependents::",('0','1','2','3+'))
      if dep=='3+':
        dep=4
      else:
        dep=int(dep)
      se_emp=st.radio("Are you Self_Employed?",("No","Yes"))
      if se_emp=="Yes":
        se_emp=1
      else:
        se_emp=0
      edu=st.radio("Are you a Graduate?",("Graduate","Not Graduate"))
      if edu=="Graduate":
        edu=0
      else:
        edu=1
      applicantincome=st.number_input("Enter your Income::")
      coapplicantincome=st.number_input("Enter your Co-Applicant Income::")
      loanamt=st.number_input("Enter the Loan Amount::")
      loanamtterm=st.number_input("Enter the Loan Amount Term::")
      credit=st.number_input("Enter your Credit History::")
      pro=st.radio("Select your Property Area::",("Urban","Rural","Semiurban"))
      if pro=="Rural":
        pro=0
      elif pro=="Semiurban":
        pro=1
      else:
        pro=2
      x=[[gender,mar,dep,edu,se_emp,applicantincome,coapplicantincome,loanamt,loanamtterm,credit,pro]]
      res=model.predict(x)
      submitted=st.form_submit_button("Submit")
      if submitted:
        if res=="Y":
          st.subheader("There is a possibility for your loan to get approved in the banks...")
        else:
          st.write("Unfortunately, there is no possibility for your loan to get approved with current bank conditions.\n You can contact us for help.")


Writing app.py
